# End-to-end Fuzzy Deduplication

GPU accelerated implementation of a MinHash-LSH based fuzzy deduplication. For more information about Fuzzy deduplication in NeMo Curator, refer to the [Deduplication](https://docs.nvidia.com/nemo/curator/latest/curate-text/process-data/deduplication/index.html) section of the documentation page.

The tutorial here shows how to run Fuzzy Duplication on text data by executing 2 end to end workflows.
These 2 workflows roughly cover the following steps to perform fuzzy deduplication:

1. Read original dataset
2. Compute MinHashes signatures of these documents
3. Perform LSH - Group Minhashes into bands/buckets and shuffle these bands/buckets so that documents in the same bucket are in the same batch/file.
4. Convert the LSH outputs (bucket_id -> doc_id mapping) into an edgelist in preparation for connected components. 
5. Compute connected components across all potential duplicates found via LSH.
6. Generate list of duplicate documents by randomly selecting 1 document to keep from each group/component and dropping the rest.
7. Remove duplicates based on the generated duplicate list.

We also allow users to also run these steps independently, which will be covered in the step by step tutorial in the same directory as this tutorial.

In [1]:
import os

import fsspec

# Silence Curator logs via Loguru
os.environ["LOGURU_LEVEL"] = "INFO"

import pandas as pd

input_dataset_path = "./input"  # Path to input dataset
fuzzy_output_dir = "./fuzzy_outputs"  # Path to store all fuzzy outputs including cache & deduped dataset

fuzzy_cache_path = os.path.join(
    fuzzy_output_dir, "cache"
)  # Path to store fuzzy deduplication intermediates (minhash, lsh etc.)
deduplicated_output_path = os.path.join(fuzzy_output_dir, "fuzzy_deduped_dataset")

input_filetype = (
    "parquet"  # this can be either of jsonl or parquet (you'll need to change how input data is generated)
)
# Note: It's important that this is constant across identification and removal.
# More information about choosing a good blocksize is mentioned in the performance considerations section below
input_blocksize = "512MiB"
output_filetype = "parquet"  # this can be either of jsonl or parquet

storage_options = None  # Optional additional cloud I/O args to pass into Pandas/cuDF during I/O operations.
fs, _ = fsspec.url_to_fs(fuzzy_cache_path, **storage_options if storage_options is not None else {})

### Downloading and saving a sample dataset

We download and save the [Tinystories](https://huggingface.co/datasets/roneneldan/TinyStories) dataset to the specified `input_dataset_path` above. This step can be skipped if running on a different dataset that's already present in the input_dataset_path.

In [2]:
from nemo_curator.utils.file_utils import get_all_file_paths_under

if len(get_all_file_paths_under(input_dataset_path, storage_options=storage_options)) == 0:
    import os
    import uuid

    from datasets import load_dataset

    input_df = load_dataset("roneneldan/TinyStories", split="train").to_pandas()
    num_rows_per_file = 10_000

    os.makedirs(input_dataset_path, exist_ok=True)

    for i, start_idx in enumerate(range(0, len(input_df), num_rows_per_file)):
        if i % 50 == 0:
            print(f"Processing file {i}")
        end_idx = min(len(input_df), start_idx + num_rows_per_file)
        subset_df = input_df.iloc[start_idx:end_idx].copy()
        subset_df["id"] = [str(uuid.uuid4()) for _ in range(len(subset_df))]
        subset_df.to_parquet(
            os.path.join(input_dataset_path, f"part_{i}.parquet"), index=False, storage_options=storage_options
        )

    print(f"Created {i + 1} files")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00004-2d5a1467fff108(…):   0%|          | 0.00/249M [00:00<?, ?B/s]

data/train-00001-of-00004-5852b56a2bd28f(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/train-00002-of-00004-a26307300439e9(…):   0%|          | 0.00/246M [00:00<?, ?B/s]

data/train-00003-of-00004-d243063613e5a0(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/validation-00000-of-00001-869c898b5(…):   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

Processing file 0
Processing file 50
Processing file 100
Processing file 150
Processing file 200
Created 212 files


## Running as a Single Stage (End-to-End)

See the [API Reference](https://docs.nvidia.com/nemo/curator/latest/apidocs/stages/stages.deduplication.fuzzy.workflow.html#api) for more information about the `FuzzyDeduplicationWorkflow` class.

### General Notes
#### ID Generation
1. The ID generation process requires a Ray cluster to be started before running the workflow either from the CLI or by using the `RayClient` API in Curator.
2. The Fuzzy Deduplication Workflow doesn't utilize any existing IDs in the input dataset and instead generates IDs on the fly using an ID Generator actor.
3. The ID Generator gives each row a unique increasing integer ID, based on the order files are read.
4. This avoids expensive ID->Integer encoding for the underlying connected components algorithm which only supports integer IDs.
5. When we find duplicates, we save these integer IDs in sorted files with multiple row groups.
6. We also save a `fuzzy_id_generator.json` which maintains a mapping of input file partitions to ID ranges for that batch.
7. During removal, reading the same file groups will give the same integer IDs, using the min/max ID values, we can find all corresponding duplicates in that range making the process faster.

#### Performance Considerations
1. LSH - Configuring `bands_per_iteration` controls how many bands to process simultaneously in a single shuffle. Higher values can lead to faster performance but might increase memory pressure.
2. A low `input_blocksize` may not saturate the GPUs enough while a high `input_blocksize` can lead to OOM errors during MinHash and excessive object store usage during removal. It's recommend to keep it at 512MiB-1.5GiB and reduce if running into OOMs during MinHash.
3. The removal step can be memory intensive and it's recommend to set a higher fraction of object store memory for removal (if the machine has enough RAM). The `RayDataExecutor` showed better results during duplicate removal.
4. The removal workflow is CPU only and can be run  on machines that don't have GPUs

#### Hyperparameter Considerations
1. The current defaults for fuzzy deduplication (260 hashes, 13 hashes per band) approximate finding documents with a Jaccard similarity of 0.8. For more information on selecting the number of bands/hashes it's recommended to analyze the S curve and tolerable threshold for false positives (and negatives). More information about LSH can be found in section `3.4.2` [here](http://infolab.stanford.edu/~ullman/mmds/ch3n.pdf).
2. The `char_ngrams` values of 24 is set to approximate roughly ngrams that correspond to ~5 words.


In [3]:
from nemo_curator.stages.deduplication.fuzzy import FuzzyDeduplicationWorkflow
from nemo_curator.stages.deduplication.id_generator import CURATOR_DEDUP_ID_STR
from nemo_curator.stages.text.deduplication import TextDuplicatesRemovalWorkflow

# All workflows support passing in different kwargs and storage_options for the read, cache and output datasets
# We use a common one here for simplicity
io_kwargs = {"storage_options": storage_options} if storage_options is not None else None

identification_workflow = FuzzyDeduplicationWorkflow(
    cache_path=fuzzy_cache_path,
    output_path=fuzzy_output_dir,
    input_path=input_dataset_path,
    input_filetype=input_filetype,
    input_blocksize=input_blocksize,
    text_field="text",
    seed=42,
    char_ngrams=24,
    minhashes_per_band=13,
    bands_per_iteration=10,
    read_kwargs=io_kwargs,
    cache_kwargs=io_kwargs,
    write_kwargs=io_kwargs,
)

removal_workflow = TextDuplicatesRemovalWorkflow(
    input_path=input_dataset_path,  # Must be identical to the path used during identification
    ids_to_remove_path=os.path.join(fuzzy_output_dir, "FuzzyDuplicateIds"),
    output_path=deduplicated_output_path,
    input_filetype=input_filetype,
    input_blocksize=input_blocksize,  # This must be identical to the blocksize used during identification
    ids_to_remove_duplicate_id_field=CURATOR_DEDUP_ID_STR,
    id_generator_path=os.path.join(fuzzy_output_dir, "fuzzy_id_generator.json"),
    output_filetype=output_filetype,
    input_kwargs=io_kwargs,  # read_kwargs for input dataset
    ids_to_remove_read_kwargs=io_kwargs,  # read_kwargs for removal_id's generated by Fuzzy workflow
    id_generator_storage_options=storage_options,
    output_kwargs=io_kwargs,
)

In [4]:
from nemo_curator.backends.experimental.ray_data import RayDataExecutor
from nemo_curator.core.client import RayClient

client = RayClient(num_cpus=64, num_gpus=2)  # change as needed
client.start()

_ = identification_workflow.run()
_ = removal_workflow.run(executor=RayDataExecutor())

2025-12-09 19:31:13.988 | WARNING  | nemo_curator.core.client:start:106 - No monitoring services are running. Please run the `start_prometheus_grafana.py` script from nemo_curator/metrics folder to setup monitoring services separately.
2025-12-09 19:31:14.044 | INFO     | nemo_curator.core.utils:init_cluster:135 - Ray start command: ray start --head --node-ip-address 127.0.1.1 --port 6380 --metrics-export-port 8081 --dashboard-host 127.0.0.1 --dashboard-port 8267 --ray-client-server-port 20000 --temp-dir /tmp/ray --disable-usage-stats --num-gpus 2 --num-cpus 64 --block
2025-12-09 19:31:14,045	INFO worker.py:1691 -- Using address 127.0.1.1:6380 set in the environment variable RAY_ADDRESS
2025-12-09 19:31:14,050	INFO worker.py:1832 -- Connecting to existing Ray cluster at address: 127.0.1.1:6380...
[2025-12-09 19:31:21,771 W 35875 35875] global_state_accessor.cc:505: Some processes that the driver needs to connect to have not registered with GCS, so retrying. Have you run 'ray start' on 

2025-12-09 19:31:16,858	INFO usage_lib.py:447 -- Usage stats collection is disabled.
2025-12-09 19:31:16,859	INFO scripts.py:914 -- Local node IP: 127.0.1.1
2025-12-09 19:31:24,784	SUCC scripts.py:950 -- --------------------
2025-12-09 19:31:24,784	SUCC scripts.py:951 -- Ray runtime started.
2025-12-09 19:31:24,784	SUCC scripts.py:952 -- --------------------
2025-12-09 19:31:24,784	INFO scripts.py:954 -- Next steps
2025-12-09 19:31:24,785	INFO scripts.py:957 -- To add another node to this Ray cluster, run
2025-12-09 19:31:24,785	INFO scripts.py:960 --   ray start --address='127.0.1.1:6380'
2025-12-09 19:31:24,785	INFO scripts.py:969 -- To connect to this Ray cluster:
2025-12-09 19:31:24,785	INFO scripts.py:971 -- import ray
2025-12-09 19:31:24,785	INFO scripts.py:972 -- ray.init(_node_ip_address='127.0.1.1')
2025-12-09 19:31:24,785	INFO scripts.py:984 -- To submit a Ray job using the Ray Jobs CLI:
2025-12-09 19:31:24,785	INFO scripts.py:985 --   RAY_API_SERVER_ADDRESS='http://127.0.0.1

/opt/venv/lib/python3.12/site-packages/ray/_private/worker.py:2051: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
2025-12-09 19:31:28.298 | INFO     | nemo_curator.pipeline.pipeline:build:70 - Planning pipeline: minhash_pipeline
2025-12-09 19:31:28,299	INFO worker.py:1691 -- Using address 127.0.1.1:6380 set in the environment variable RAY_ADDRESS
2025-12-09 19:31:28,303	INFO worker.py:1832 -- Connecting to existing Ray cluster at address: 127.0.1.1:6380...
2025-12-09 19:31:28,309	INFO worker.py:2003 -- Connected to Ray cluster. View the dashboard at http://127.0.0.1:8267 
2025-12-09 19:31:28.327 | INFO     | nemo_curator.backends.experimental.utils:execute_setup_on_node:120 - Executing setup on node 6a76449a01128728abfd16efd066646e2a324805bc09b95fd625caca for 2 stages

Running 0: 0.00 row [00:00, ? row/s]

- MapBatches(FilePartitioningStageTask) 1: 0.00 row [00:00, ? row/s]

- StreamingRepartition 2: 0.00 row [00:00, ? row/s]

- MapBatches(ParquetReaderStageActor) 3: 0.00 row [00:00, ? row/s]

- MapBatches(TextDuplicatesRemovalStageTask)->MapBatches(ParquetWriterTask) 4: 0.00 row [00:00, ? row/s]

2025-12-09 19:33:40,108	WARNING resource_manager.py:134 -- ⚠️  Ray's object store is configured to use only 10.2% of available memory (186.3GiB out of 1824.0GiB total). For optimal Ray Data performance, we recommend setting the object store to at least 50% of available memory. You can do this by setting the 'object_store_memory' parameter when calling ray.init() or by setting the RAY_DEFAULT_OBJECT_STORE_MEMORY_PROPORTION environment variable.
(MapBatches(FilePartitioningStageTask) pid=49184) 2025-12-09 19:33:40.352 | INFO     | nemo_curator.stages.file_partitioning:process:95 - Found 212 files
(MapBatches(FilePartitioningStageTask) pid=49184) 2025-12-09 19:33:40.352 | INFO     | nemo_curator.stages.file_partitioning:process:132 - Created 2 file groups from 212 files
2025-12-09 19:34:02,812	INFO streaming_executor.py:298 -- ✔️  Dataset dataset_5_0 execution finished in 22.78 seconds
2025-12-09 19:34:02,818	INFO util.py:257 -- Exiting prefetcher's background thread
2025-12-09 19:34:02.8

### Looking at Intermediate Results and Output

#### MinHash Results
1. `_curator_dedup_id` - The IDs assigned to this dataset on the fly during the initial read.
2. `_minhash_signature` - MinHash Signature

#### LSH Results
1. `_bucket_id` - The bucket/band identifier
2. `_curator_dedup_id` - List of all document IDs that belong to that bucket

#### Buckets To Edges Result
1. `_curator_dedup_id_x`, `_curator_dedup_id_y` - Mapping of edges in a Graph where each column are documents that are potential duplicates.

In [5]:
minhash_path = os.path.join(fuzzy_cache_path, "MinHashStage")
display(pd.read_parquet(fs.unstrip_protocol(fs.find(minhash_path)[0]), storage_options=storage_options).head())

lsh_path = os.path.join(fuzzy_cache_path, "LSHStage")
display(pd.read_parquet(fs.unstrip_protocol(fs.find(lsh_path)[0]), storage_options=storage_options).head())

b2e_path = os.path.join(fuzzy_cache_path, "BucketsToEdgesStage")
display(pd.read_parquet(fs.unstrip_protocol(fs.find(b2e_path)[0]), storage_options=storage_options).head())

,_curator_dedup_id,_minhash_signature
0,1110000,"[7605662, 4337118, 6484429, 8346452, 6444847, ..."
1,1110001,"[2938900, 4353439, 3290251, 1074973, 4860283, ..."
2,1110002,"[129864, 3231856, 1046776, 1777482, 4032304, 1..."
3,1110003,"[8395592, 14856053, 2666175, 3533855, 18487142..."
4,1110004,"[265861, 19780184, 11961285, 16232798, 1259680..."


,_bucket_id,_curator_dedup_id
0,b0_000055fd7daae1e46223e8b7e06bf2e0,"[1178375, 2079489]"
1,b0_0000f975e5bcda25838df43b0d37737f,"[965853, 1334885]"
2,b0_0001c9dff36e10d709d64123cb0dee4d,"[299769, 1936007]"
3,b0_00020b2c889483bd6a78ffe9a8d7deb1,"[261169, 2008278]"
4,b0_00024c2b7321353410dd908eb31499bd,"[213076, 990707]"


,_curator_dedup_id_x,_curator_dedup_id_y
0,128422,1370422
1,568235,1552660
2,873929,2098065
3,997647,1289916
4,1309916,1952254


#### Connected Components Result

1. `_curator_dedup_id` - The document IDs
2. `_duplicate_group_id` - The group ID that document belongs to. Documents with the same duplicate group ID are duplicates

In [6]:
cc_path = os.path.join(fuzzy_cache_path, "ConnectedComponentsStage")
cc_df = pd.read_parquet(cc_path, storage_options=storage_options)  # works with pandas since the input here is small
display(cc_df)
grouped_cc_df = cc_df.groupby("_duplicate_group_id")._curator_dedup_id.agg(list)
display(grouped_cc_df)
duplicate_cluster_sizes = cc_df._duplicate_group_id.value_counts()
display(duplicate_cluster_sizes)

,_curator_dedup_id,_duplicate_group_id
0,580,482276
1,582,320167
2,588,482280
3,590,320169
4,592,320170
...,...,...
640509,2119663,480120
640510,2119665,320162
640511,2119668,320163
640512,2119672,158947


_duplicate_group_id
0              [576, 187440]
2              [578, 187442]
3              [579, 187443]
4              [581, 187445]
6              [584, 187448]
                 ...        
640508    [2119667, 1942716]
640509    [2119669, 1942718]
640511    [1942720, 2119671]
640512    [1942722, 2119673]
640513    [2119675, 1942724]
Name: _curator_dedup_id, Length: 320043, dtype: object

_duplicate_group_id
142559    230
505715      3
436273      3
88121       3
88124       3
         ... 
551430      2
426966      2
551428      2
426964      2
159712      2
Name: count, Length: 320043, dtype: int64

Based on the distribution above we can see that there is one cluster/group where 230 documents are all duplicates followed by many smaller clusters with 2 or 3 documents that are duplicates.

#### FuzzyDuplicateIds Results (List of duplicate docs to remove)
1. `_curator_dedup_id` - ID of docs in the removal list

In [7]:
duplicate_ids_path = os.path.join(fuzzy_output_dir, "FuzzyDuplicateIds")
duplicates_df = pd.read_parquet(duplicate_ids_path, storage_options=storage_options)
display(duplicates_df.head())

print(f"Number of duplicate documents found for removal: {len(duplicates_df)}")

,_curator_dedup_id
0,594
1,607
2,614
3,625
4,632


Number of duplicate documents found for removal: 320471


#### Checking that the duplicate ids list contains only one document per group

In [8]:
# As an example let's look at the group with the largest number of duplicates
largest_duplicate_cluster = grouped_cc_df.loc[duplicate_cluster_sizes.index[0]]

# number of docs in the removal list from this group
docs_to_remove_in_group = duplicates_df._curator_dedup_id.isin(largest_duplicate_cluster).sum()

print(f"Number of documents in the duplicate group: {len(largest_duplicate_cluster)}")
print(f"Number of documents in the removal list from the same group: {docs_to_remove_in_group}")
assert docs_to_remove_in_group == (len(largest_duplicate_cluster) - 1)  # noqa: S101

Number of documents in the duplicate group: 230
Number of documents in the removal list from the same group: 229


#### Advanced: Looking at examples of duplicate documents

1. This analysis involves re-reading the input data with the same ID mapping that was used during duplicate identification.
2. Merging the input data with the connected components results on the `_curator_dedup_id` column to associate each document which the duplicate group it belongs to which can be used for further analysis.

**NOTE**: This analysis approach is intended as an example for smaller datasets and only works for cases where the connected components dataframe is small and fits comfortable in memory. It is not recommended for larger datasets.

In [9]:
from nemo_curator.pipeline import Pipeline
from nemo_curator.stages.base import ProcessingStage
from nemo_curator.stages.resources import Resources
from nemo_curator.stages.text.io.reader import JsonlReader, ParquetReader
from nemo_curator.tasks.document import DocumentBatch


class CustomMergeStage(ProcessingStage[DocumentBatch, DocumentBatch]):
    """
    Warning: This should not be attempted with large connected components results.
    A small stage that merges the input data (using the id's generated) with the connected components result.
    Works because CC results are small enough to fit per batch.
    """

    resources = Resources(cpus=1.0)

    def process(self, batch: DocumentBatch) -> DocumentBatch:
        df = batch.to_pandas().merge(cc_df, how="inner", on=[CURATOR_DEDUP_ID_STR])
        return DocumentBatch(
            task_id=batch.task_id, dataset_name=batch.dataset_name, data=df, _stage_perf=batch._stage_perf
        )


ReaderClass = ParquetReader if input_filetype == "parquet" else JsonlReader
pipeline = Pipeline(
    name="Explore duplicates",
    stages=[
        ReaderClass(file_paths=input_dataset_path, blocksize=input_blocksize, _assign_ids=True, read_kwargs=io_kwargs),
        CustomMergeStage(),
    ],
)

In [ ]:
from nemo_curator.core.client import RayClient
from nemo_curator.stages.deduplication.id_generator import create_id_generator_actor, kill_id_generator_actor

try:
    create_id_generator_actor(
        filepath=os.path.join(fuzzy_output_dir, "fuzzy_id_generator.json"), storage_options=storage_options
    )
    merged_results = pipeline.run()
    merged_df = pd.concat([batch.to_pandas() for batch in merged_results]).sort_values("_duplicate_group_id")
finally:
    kill_id_generator_actor()

In [11]:
display(merged_df[merged_df._curator_dedup_id.isin(largest_duplicate_cluster)])

,text,id,_curator_dedup_id,_duplicate_group_id
289502,,1084d700-f756-4dca-a198-f3b6414909e1,952302,142559
200157,,777cc06f-29fc-47ef-ae25-27a1ee7af9f8,1795067,142559
92644,,900e74e2-b609-4fb8-be0c-0ecb2c593cc6,305150,142559
200156,,662b6000-0f0c-465a-b075-c7dfcb403f91,1795066,142559
200155,,577a7853-11d8-4a35-a96f-2a3725c5f446,1795065,142559
...,...,...,...,...
330564,,c7cf48e1-0053-47a3-8a2b-8f66ad659fbd,1083898,142559
31037,,c92fe911-f72f-4e01-b609-fb2801eafd75,1214995,142559
30803,,cf7e6d4b-ed55-4c50-96d1-8b25de87bed6,1213140,142559
95464,,9bb7152f-eb03-477e-b77b-92fc064903d7,1437250,142559


The largest cluster/group of duplicates in this dataset seems to be all documents with empty/no text.

Let's look at the second largest cluster of documents.

In [12]:
duplicates = merged_df[merged_df._curator_dedup_id.isin(grouped_cc_df.loc[duplicate_cluster_sizes.index[1]])]
display(duplicates)

print(f"\nDocument1\n----------\n{duplicates.iloc[0].text}")
print(f"\nDocument2\n----------\n{duplicates.iloc[1].text}")

,text,id,_curator_dedup_id,_duplicate_group_id
255165,"Once upon a time, there was a band of animals....",301ec69c-e5ef-4da8-81bd-b9d426526ef3,843163,505715
170413,"Once upon a time, there was a band of animals....",b2412c63-121e-4579-b881-4a922ccc4b08,561487,505715
32340,"Once upon a time, there was a band of animals....",8cb47acc-0f61-4da2-b88c-02f7fbdb0b21,99569,505715



Document1
----------
Once upon a time, there was a band of animals. They were all friends and loved to play music together. The band had a cat who played the guitar, a dog who played the drums, and a bird who sang.

One day, the band got into a big fight. The cat was impatient and wanted to play a fast song, but the dog wanted to play a slow song. They argued and argued until they couldn't play together anymore.

The bird missed playing with the band and decided to do something about it. She went to each animal and talked to them. She told the cat to be patient and let the dog play a slow song. She told the dog to try playing a faster beat. Finally, the band united and played the best song they ever played before. They were all happy and played together every day after that.

Document2
----------
Once upon a time, there was a band of animals. They were all friends and loved to play music together. The band had a cat who played the guitar, a dog who played the drums, and a bird who san

In [13]:
client.stop()

2025-12-09 20:16:52.778 | INFO     | nemo_curator.core.client:stop:181 - NeMo Curator has stopped the Ray cluster it started by killing the Ray GCS process. It is advised to wait for a few seconds before running any Ray commands to ensure Ray can cleanup other processes.If you are seeing any Ray commands like `ray status` failing, please ensure /tmp/ray/ray_current_cluster has correct information.


### Conclusion
We were able to find and remove ~320_000 duplicate documents in a dataset of ~2.1 Million Rows 